# QMUL Careers Team Python Script
This is a detailed guide on how to run this programme.


## Excel File Housekeeping



We need the excel workbook that the programme will use, and within that the excel spreadsheet with the student information. 

Please name the file: <b>'applications.xlsx'</b> with the application information on a spreadsheet called: <b>data </b> and save it on your computer.


It is really important that the column headers which are used for sorting are named correctly. Before you upload the spreadsheet please ensure these columns match these headers, you can paste them into the spreadsheet as they appear below:

student number <br>
year of study<br>
gender<br>
faculty<br>
total <i> (referring to total score) </i><br>
course<br>
personality type<br>

The video below shows how to add this to the programme.

Each text box describe what the code does in the cell below it.

The first cell imports some of the software that the programme needs, if you click the small play button above, or press Shift and Enter, it will run the cell.

In [1]:
import pandas as pd
import openpyxl
import numpy as np
import os
import ipywidgets

In [2]:
from IPython.display import Video; from ipywidgets import interactive, IntSlider

vid = Video(filename="media/add_applications.mov",data="", width=800, height = 600)
display(vid)

Next, point the programme at the excel sheet just uploaded.

In [3]:
def get_sheet():
    for file in os.listdir('.'):
        if file.endswith("applications.xlsx"):
            myfile = file
            print(f'{file} has been selected.')
        else:
            pass           
    try:
        return myfile
    except:
        print('No file has been uploaded')
    
filename = get_sheet()

applications.xlsx has been selected.


Read the spreadsheet in, and perform some data cleaning.

In [4]:
df = pd.read_excel(filename,"data")
for i in df.columns:
    df.rename(columns={i:i.lower()}, inplace=True)

criteria = [ 
'student number',
'year of study',
'gender',
'faculty',
'total',
'course',
'personality type']

Please use the dropdown menu to select the criteria to sort the groups.

In [5]:
mywidge1 = ipywidgets.widgets.SelectMultiple(
    options=criteria,
    value=['total'],
    rows=len(criteria),
    disabled=False
)

class SelectMultipleInteract(ipywidgets.widgets.HBox):

    def __init__(self):
        self.W1 = ipywidgets.widgets.SelectMultiple(
        options=criteria,
        value=['total'],
        rows=len(criteria),
        disabled=False
)

        self.selectors = [self.W1]
        super().__init__(children=self.selectors)
        self._set_observes()

    def _set_observes(self):
        for widg in self.selectors:
            widg.observe(self._observed_function, names='value')

    def _observed_function(self, widg):
        for widg in self.selectors:
            # print(widg.get_interact_value())
            return list(widg.get_interact_value())

mywidge = SelectMultipleInteract()
mywidge

SelectMultipleInteract(children=(SelectMultiple(index=(4,), options=('student number', 'year of study', 'gende…

In [6]:
options = mywidge._observed_function(mywidge1)
for_sorting = ['total']
for_sorting = for_sorting+[x for x in options if x != 'total']
print(f'The algorithm will sort using {for_sorting} from left to right')

The algorithm will sort using ['total'] from left to right


Any missing scores will be replaced with the average score of the dataset.

In [7]:
df['total'].fillna(df['total'].mean(), inplace=True)

Some exploratory data analysis in the next 2 cells

In [8]:
df['gender'].value_counts()

F    42
M    18
Name: gender, dtype: int64

In [9]:
df['year of study'].value_counts()

2    32
3    20
1     5
4     3
Name: year of study, dtype: int64

Take only the columns important to establishing the groups. 

In [10]:
group_df = df[criteria]

In [11]:
group_df = group_df.assign(groupno='')



## The Sorting Algorithm

In [12]:
def assign_group(df):
    df.sort_values(by=for_sorting, ascending=False, inplace=True)
    count = 1
    up = True
    for i,row in group_df.iterrows():

        if count <= round(len(df)/5,0) and up == True:
            df.at[i,'groupno'] = count
            count += 1

        elif count > round(len(df)/5,0) and up == True:
            df.at[i,'groupno'] = count-1
            count-=1
            up = False

        if count > 1 and up == False:
            df.at[i,'groupno'] = count
            count -= 1
            
        elif count == 1 and up == False:
            df.at[i,'groupno'] = count
            up = True
    
    return df

assigned = assign_group(group_df)

    

## Data Analysis

In [13]:
assigned.head()

,student number,year of study,gender,faculty,total,course,personality type,groupno
25,200439264,2,F,Humanities and Social Sciences,12.0,BA FT English,Advocate INFJ,1
12,200463971,2,F,Humanities and Social Sciences,11.5,BA FT Politics with Business Management,Protagonist ENFJ,2
44,180291371,3,M,Humanities and Social Sciences,11.5,LLB FT Law,Commander ENTJ,3
23,200222002,2,F,Science and Engineering,11.5,BSc FT Biomedical Sciences,Consul ESFJ,4
26,200429597,2,F,Humanities and Social Sciences,11.0,BSc FT Business Management,Campaigner ENFP,5


In [14]:
assigned.groupby(['groupno'])[['total']].mean().round(2).min()

total    6.9
dtype: float64

In [15]:
assigned.pivot_table(index=['groupno'], columns=['year of study'], aggfunc='size', fill_value=0)


year of study,1,2,3,4
groupno,,,,
1,2,2,1,0
2,0,4,1,0
3,0,2,3,0
4,0,4,0,1
5,2,2,1,0
6,1,3,1,0
7,0,0,5,0
8,0,4,1,0
9,0,1,4,0


In [16]:
assigned.pivot_table(index=['groupno'], columns=['gender'], aggfunc='size', fill_value=0)


gender,F,M
groupno,,
1,4,1
2,3,2
3,3,2
4,2,3
5,4,1
6,4,1
7,3,2
8,2,3
9,5,0


### Create a new spreadsheet to download

In [17]:
subset = assigned[['student number','groupno']]

final = pd.merge(df,subset, how='inner', on='student number')



In [22]:
final.to_excel("grouped_student_applications.xlsx",sheet_name='Grouped Students') 

In [35]:
import urllib.parse
def build_csv_data(dataframe):
    csv_data = dataframe.to_csv(index=False, encoding='utf-8')
    return csv_data

In [37]:
from ipywidgets import Output, Button
from IPython.display import HTML, clear_output
from base64 import b64encode

download_output = Output()
display(download_output)
def trigger_download(filename, kind='text/csv'):
    # see https://developer.mozilla.org/en-US/docs/Web/HTTP/Basics_of_HTTP/Data_URIs for details
    text = build_csv_data(final)
    content_b64 = b64encode(text.encode()).decode()
    data_url = f'data:{kind};charset=utf-8;base64,{content_b64}'
    js_code = f"""
        var a = document.createElement('a');
        a.setAttribute('download', '{filename}');
        a.setAttribute('href', '{data_url}');
        a.click()
    """
    with download_output:
        clear_output()
        display(HTML(f'<script>{js_code}</script>'))

btn = Button(description='Download Groups')
def download_groups(e=None):
    trigger_download('applications_grouped.csv', kind='text/csv')
btn.on_click(download_groups)
display(btn)

Output()

Button(description='Download Groups', style=ButtonStyle())

In [ ]:
from IPython.display import Video; from ipywidgets import interactive, IntSlider

vid = Video(filename="media/download_file.mov",data="", width=800, height = 600)
display(vid)

## Make sure you download your new spreadsheet, you can copy and paste it into your previous working document.